In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import torch
import os
import sys
sys.path.append("../")

In [2]:
atlas_data_dir = "/data/cb/scratch/datasets/atlas"
atlas_dynamicslabels_dir = "/data/cb/scratch/datasets/atlas_dynamics_labels"
data_config_dir = "../trained/configs/data"

In [10]:
import yaml
from pathlib import Path

# Load YAML config
with open(data_config_dir+"/atlas_config.yaml", "r") as file:
    config = yaml.safe_load(file)
config

{'repo_dir': '/data/cb/mihirb14/projects/DynaProt',
 'struc_data_dir': '/data/cb/scratch/datasets/pdb_npz',
 'dynam_data_dir': '/data/cb/scratch/datasets/atlas_dynamics_labels',
 'max_seq_len': 512}

In [11]:
from DynaProt.data.datasets import DynaProtDataset, OpenFoldBatchCollator

dataset = DynaProtDataset(config)
dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=10,
        collate_fn=OpenFoldBatchCollator(),
        num_workers=12,
        shuffle=False,
    )

batch_prots = next(iter(dataloader))

for k in batch_prots.keys():
    # print(k,batch_prots[k])
    print(f"{k}: {batch_prots[k].shape}")

aatype:torch.Size([10, 512, 21])
residue_index:torch.Size([10, 512])
all_atom_positions:torch.Size([10, 512, 37, 3])
all_atom_mask:torch.Size([10, 512, 37])
resi_pad_mask:torch.Size([10, 512])
dynamics_means:torch.Size([10, 512, 3])
dynamics_vars:torch.Size([10, 512, 3, 3])
frames:torch.Size([10, 512, 4, 4])
